# Score LOL
### Modelo de avaliação de players por liga/ano
---
**Como usar:** Altere apenas as variáveis na célula de CONFIGURAÇÃO abaixo e rode todas as células em ordem.

In [ ]:
# ============================================================
# CÉLULA 1 — CONFIGURAÇÃO
# Mude aqui para testar qualquer liga/ano
# ============================================================

CAMINHO_CSV = '/content/drive/MyDrive/projeto_cblol/2026_LoL_esports_match_data_from_OraclesElixir.csv'
LIGA = 'CBLOL'  # ex: 'LCK', 'LPL', 'LCS', 'CBLOL', 'LTA S'
TIMES_FILTRO = None  # None = todos os times da liga
                     # ou lista ex: ['T1', 'Gen.G', 'DRX']
DATA_PLAYOFFS_MANUAL = '2026-02-02'  # None = usa coluna playoffs do CSV
                             # ou ex: '2026-02-02' para marcar manualmente
UNIFICAR_TIMES = None  # None = sem unificação
                       # ou dict ex: {'Isurus Estral': 'Isurus'}

In [ ]:
# ============================================================
# CÉLULA 2 — SETUP E CARREGAMENTO
# ============================================================
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import warnings
warnings.filterwarnings('ignore')

df_raw = pd.read_csv(CAMINHO_CSV, low_memory=False)
df = df_raw[(df_raw['league'] == LIGA) & (df_raw['position'] != 'team')].copy()

if UNIFICAR_TIMES:
    df['teamname'] = df['teamname'].replace(UNIFICAR_TIMES)

if TIMES_FILTRO:
    df = df[df['teamname'].isin(TIMES_FILTRO)].copy()

if DATA_PLAYOFFS_MANUAL:
    df['playoffs'] = (df['date'] >= DATA_PLAYOFFS_MANUAL).astype(int)

# Playoffs: vitória pesa 1.5x, derrota pesa 1.2x, fase de grupos 1.0x
df['peso_jogo'] = df.apply(
    lambda x: 1.5 if (x['playoffs'] == 1 and x['result'] == 1)
    else 1.2 if (x['playoffs'] == 1 and x['result'] == 0)
    else 1.0, axis=1
)

print(f"Liga: {LIGA}")
print(f"Times: {df['teamname'].nunique()} | {sorted(df['teamname'].unique())}")
print(f"Jogos únicos: {df['gameid'].nunique()}")
print(f"Jogos por role: {df.groupby('position').size().to_dict()}")
print(f"Pesos: {df['peso_jogo'].value_counts().to_dict()}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Liga: CBLOL
Times: 8 | ['FURIA', 'Fluxo W7M', 'LOUD', 'Leviatan', 'LØS', 'RED Canids', 'Vivo Keyd Stars', 'paiN Gaming']
Jogos únicos: 61
Jogos por role: {'bot': 122, 'jng': 122, 'mid': 122, 'sup': 122, 'top': 122}
Pesos: {1.0: 280, 1.5: 165, 1.2: 165}


In [ ]:
# ============================================================
# CÉLULA 3 — MÉTRICAS DERIVADAS E FUNÇÕES
# ============================================================

df['kp'] = (df['kills'] + df['assists']) / df['teamkills'].replace(0, 1)
df['ks%'] = df['kills'] / df['teamkills'].replace(0, 1)
df['dmg_efficiency'] = df['damageshare'] / df['earnedgoldshare'].replace(0, 0.01)
df['combat_efficiency'] = df['dpm'] * (df['kills'] + df['assists'] + 1) / (df['deaths'] + 1)
df['delta_gold_10_15'] = df['golddiffat15'] - df['golddiffat10']
df['delta_gold_15_20'] = df['golddiffat20'] - df['golddiffat15']
df['delta_gold_20_25'] = (df['golddiffat25'] - df['golddiffat20']).fillna(0)
df['delta_xp_10_15'] = df['xpdiffat15'] - df['xpdiffat10']
df['delta_xp_15_20'] = df['xpdiffat20'] - df['xpdiffat15']
df['delta_xp_20_25'] = (df['xpdiffat25'] - df['xpdiffat20']).fillna(0)
df['delta_cs_10_15'] = df['csdiffat15'] - df['csdiffat10']
df['delta_cs_15_20'] = df['csdiffat20'] - df['csdiffat15']
df['delta_cs_20_25'] = (df['csdiffat25'] - df['csdiffat20']).fillna(0)

features_final = [
    'golddiffat10', 'delta_gold_10_15', 'delta_gold_15_20', 'delta_gold_20_25',
    'xpdiffat10', 'delta_xp_10_15', 'delta_xp_15_20', 'delta_xp_20_25',
    'csdiffat10', 'delta_cs_10_15', 'delta_cs_15_20', 'delta_cs_20_25',
    'dmg_efficiency', 'combat_efficiency', 'kp', 'ks%',
    'cspm', 'vspm', 'firstbloodkill', 'firstbloodassist'
]

def detectar_tipo_feature(serie, resultado, n_quintis=5):
    df_temp = pd.DataFrame({'feat': serie, 'result': resultado})
    df_temp['quintil'] = pd.qcut(df_temp['feat'], q=n_quintis, labels=False, duplicates='drop')
    wr = df_temp.groupby('quintil')['result'].mean()
    otimo = wr.idxmax()
    corr = wr.reset_index()['result'].corr(pd.Series(range(len(wr))))
    if corr > 0.7:
        return 'linear_up', otimo
    elif corr < -0.7:
        return 'linear_down', otimo
    else:
        return 'sweetspot', otimo

def score_feature(serie, resultado, n_quintis=5):
    df_temp = pd.DataFrame({'feat': serie, 'result': resultado})
    df_temp['quintil'] = pd.qcut(df_temp['feat'], q=n_quintis, labels=False, duplicates='drop')
    tipo, otimo = detectar_tipo_feature(serie, resultado, n_quintis)
    n = n_quintis - 1
    if tipo == 'linear_up':
        return (serie - serie.min()) / (serie.max() - serie.min() + 0.01)
    elif tipo == 'linear_down':
        return 1 - (serie - serie.min()) / (serie.max() - serie.min() + 0.01)
    else:
        return df_temp['quintil'].apply(lambda q: 1 - abs(q - otimo) / n if pd.notna(q) else 0.5)

print('Métricas e funções prontas!')

Métricas e funções prontas!


In [ ]:
# ============================================================
# CÉLULA 4 — SCORE BASE POR JOGO
# ============================================================
scores_por_jogo = []
pesos_por_role = {}

for role in ['top', 'jng', 'mid', 'bot', 'sup']:
    df_role = df[df['position'] == role].copy()

    correlacoes = {}
    for feat in features_final:
        try:
            corr, _ = spearmanr(df_role[feat], df_role['result'])
            correlacoes[feat] = abs(corr)
        except:
            correlacoes[feat] = 0
    total = sum(correlacoes.values())
    pesos_por_role[role] = {k: v/total for k, v in correlacoes.items()}

    scores_features = {}
    for feat in features_final:
        try:
            s = score_feature(df_role[feat], df_role['result'])
            if s.isna().sum() == 0:
                scores_features[feat] = s
        except:
            pass

    df_scores = pd.DataFrame(scores_features, index=df_role.index)
    feats_ok = list(scores_features.keys())
    total_peso = sum(pesos_por_role[role][f] for f in feats_ok)
    df_role['score_jogo'] = sum(
        df_scores[feat] * (pesos_por_role[role][feat] / total_peso)
        for feat in feats_ok
    )
    scores_por_jogo.append(df_role[['playername', 'teamname', 'position', 'result', 'score_jogo', 'peso_jogo', 'earnedgoldshare']])

df_jogos = pd.concat(scores_por_jogo)

ranking = df_jogos.groupby(['playername', 'teamname', 'position']).apply(
    lambda x: pd.Series({
        'W': np.average(x['result'], weights=x['peso_jogo']),
        'Score': np.average(x['score_jogo'], weights=x['peso_jogo'])
    })
).reset_index()

print(f'Score base calculado para {len(ranking)} players')
print(ranking.sort_values('Score', ascending=False).head(5)[['playername', 'teamname', 'position', 'Score']].round(3).to_string())

Score base calculado para 43 players
   playername    teamname position  Score
31       Tatu       FURIA      jng  0.597
39        fNb  RED Canids      top  0.547
36       Zest         LØS      top  0.547
42       xyno        LOUD      top  0.542
17       JoJo       FURIA      sup  0.537


In [ ]:
# ============================================================
# CÉLULA 5 — ANCHOR SCORE
# Mede o quanto o player se destaca no próprio time
# ajustado pela força do time. Base 0.5, range 0-1.
# ============================================================
team_stats = ranking.groupby('teamname').agg(
    team_avg=('Score', 'mean'),
    team_std=('Score', 'std'),
    team_wr=('W', 'mean')
).reset_index()

ranking = ranking.merge(team_stats, on='teamname')
ranking['team_strength'] = (ranking['team_wr'] - ranking['team_wr'].min()) / (ranking['team_wr'].max() - ranking['team_wr'].min() + 0.01)
ranking['player_vs_team'] = (ranking['Score'] - ranking['team_avg']) / (ranking['team_std'] + 0.01)

jogos_count = df_jogos.groupby('playername').size().reset_index(name='n_jogos')
max_jogos = jogos_count['n_jogos'].max()
ranking = ranking.merge(jogos_count, on='playername')
ranking['confianca'] = ranking['n_jogos'] / max_jogos

ranking['anchor_raw'] = ranking['player_vs_team'] * (1 - ranking['team_strength'] * 0.3)
anchor_min = ranking['anchor_raw'].min()
anchor_max = ranking['anchor_raw'].max()
ranking['anchor_norm'] = (ranking['anchor_raw'] - anchor_min) / (anchor_max - anchor_min + 0.01)
ranking['AnchorScore'] = (
    ranking['anchor_norm'] * ranking['confianca'] +
    0.5 * (1 - ranking['confianca'])
).clip(0, 1)

print('AnchorScore calculado!')
print(ranking[['playername', 'teamname', 'Score', 'AnchorScore']].sort_values('AnchorScore', ascending=False).head(5).round(3).to_string())

AnchorScore calculado!
   playername         teamname  Score  AnchorScore
42       xyno             LOUD  0.542        0.861
31       Tatu            FURIA  0.597        0.843
12       Enga         Leviatan  0.488        0.769
36       Zest              LØS  0.547        0.764
18    Kaiwing  Vivo Keyd Stars  0.514        0.753


In [ ]:
# ============================================================
# CÉLULA 6 — CARRY SCORE
# Base 0.5. Quem recebe recursos e converte em vitória sobe.
# Quem recebe recursos e não converte cai.
# Quem não recebe recursos fica próximo de 0.5.
# ============================================================
carry_data = df_jogos.groupby('playername').apply(
    lambda x: pd.Series({
        'jogos_referencia': (x['score_jogo'] > x['score_jogo'].mean()).sum(),
        'referencias_convertidas': ((x['score_jogo'] > x['score_jogo'].mean()) & (x['result'] == 1)).sum(),
        'total_jogos': len(x)
    })
).reset_index()
carry_data['carry_rate'] = carry_data['referencias_convertidas'] / (carry_data['jogos_referencia'] + 0.01)

earn_gold_role = df_jogos.groupby(['playername', 'position'])['earnedgoldshare'].mean().reset_index()
earn_gold_role['gold_norm'] = earn_gold_role.groupby('position')['earnedgoldshare'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min() + 0.01)
)
carry_data = carry_data.merge(earn_gold_role[['playername', 'gold_norm']], on='playername', how='left').fillna(0.5)
carry_data['carry_score'] = (0.5 + carry_data['gold_norm'] * (carry_data['carry_rate'] - 0.5)).clip(0, 1)

print('Carry Score calculado!')
print(carry_data[['playername', 'gold_norm', 'carry_rate', 'carry_score']].sort_values('carry_score', ascending=False).head(5).round(3).to_string())

Carry Score calculado!
   playername  gold_norm  carry_rate  carry_score
31       Tatu      0.775       0.999        0.886
16       Hauz      0.760       0.995        0.876
12       Enga      0.689       0.899        0.775
5        Bull      0.527       0.999        0.763
28     Rabelo      0.515       0.997        0.756


In [ ]:
# ============================================================
# CÉLULA 7 — CONSISTENCY SCORE
# Mede estabilidade pelo piso de performance.
# Score alto + piso alto + pouca variação = melhor.
# Score baixo + piso baixo + muita variação = pior.
# ============================================================
consistencia = df_jogos.groupby(['playername', 'position']).apply(
    lambda x: pd.Series({
        'score_medio': np.average(x['score_jogo'], weights=x['peso_jogo']),
        'score_piso': x['score_jogo'].quantile(0.25),
        'score_teto': x['score_jogo'].quantile(0.75),
        'total_jogos': len(x)
    })
).reset_index().fillna(0)

consistencia['amplitude'] = consistencia['score_teto'] - consistencia['score_piso']

consistencia['piso_norm'] = consistencia.groupby('position')['score_piso'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min() + 0.01)
)
consistencia['amplitude_norm'] = consistencia.groupby('position')['amplitude'].transform(
    lambda x: 1 - (x - x.min()) / (x.max() - x.min() + 0.01)
)
consistencia['medio_norm'] = consistencia.groupby('position')['score_medio'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min() + 0.01)
)

consistencia['consistency_score'] = (
    consistencia['medio_norm'] * 0.4 +
    consistencia['piso_norm'] * 0.4 +
    consistencia['amplitude_norm'] * 0.2
)

print('Consistency Score calculado!')
print(consistencia[['playername', 'position', 'score_medio', 'score_piso', 'amplitude', 'consistency_score']]
      .sort_values(['position', 'consistency_score'], ascending=[True, False])
      .head(10).round(3).to_string())

Consistency Score calculado!
   playername position  score_medio  score_piso  amplitude  consistency_score
28     Rabelo      bot        0.478       0.450      0.024              0.877
33    Trigger      bot        0.509       0.454      0.132              0.814
1         Ayu      bot        0.502       0.447      0.135              0.775
11     Duduhh      bot        0.471       0.416      0.121              0.653
37        ceo      bot        0.473       0.396      0.128              0.599
2         BAO      bot        0.431       0.374      0.068              0.532
24  Morttheus      bot        0.473       0.384      0.168              0.516
5        Bull      bot        0.467       0.387      0.167              0.513
22     Marvin      bot        0.333       0.301      0.032              0.189
31       Tatu      jng        0.597       0.514      0.138              0.902


In [ ]:
# ============================================================
# CÉLULA 8 — PESOS POR CORRELAÇÃO COM VITÓRIA
# Calcula automaticamente o peso de cada score por role
# baseado na correlação com vitória (Spearman).
# ============================================================

# Junta todos os scores para calcular correlação
ranking_temp = ranking.merge(
    consistencia[['playername', 'position', 'consistency_score']],
    on=['playername', 'position'], how='left'
).fillna(0)

df_scores_jogo = df_jogos.merge(
    carry_data[['playername', 'carry_score']], on='playername', how='left'
).merge(
    ranking_temp[['playername', 'position', 'AnchorScore', 'consistency_score']],
    on=['playername', 'position'], how='left'
)

pesos_role_scores = {}
print('PESOS POR ROLE (baseados em correlação com vitória):\n')
for role in ['top', 'jng', 'mid', 'bot', 'sup']:
    df_role = df_scores_jogo[df_scores_jogo['position'] == role].copy()
    corrs = {}
    for score in ['score_jogo', 'carry_score', 'consistency_score']:
        try:
            corr, _ = spearmanr(df_role[score].fillna(0.5), df_role['result'])
            corrs[score] = abs(corr)
        except:
            corrs[score] = 0
    total = sum(corrs.values())
    pesos_role_scores[role] = {k: round(v/total, 3) for k, v in corrs.items()}
    print(f"{role.upper()}: score={pesos_role_scores[role]['score_jogo']:.1%} | carry={pesos_role_scores[role]['carry_score']:.1%} | consistency={pesos_role_scores[role]['consistency_score']:.1%}")

PESOS POR ROLE (baseados em correlação com vitória):

TOP: score=53.2% | carry=23.1% | consistency=23.7%
JNG: score=64.0% | carry=16.8% | consistency=19.1%
MID: score=74.4% | carry=3.0% | consistency=22.7%
BOT: score=66.7% | carry=17.5% | consistency=15.8%
SUP: score=70.8% | carry=22.0% | consistency=7.2%


In [ ]:
# ============================================================
# CÉLULA 9 — SCORE FINAL E RANKINGS
# Score = 95% distribuído pelos pesos calculados
# AnchorScore = 5% fixo (ajuste contextual)
# ============================================================

# Junta tudo no ranking final
ranking_final = ranking.merge(
    consistencia[['playername', 'position', 'consistency_score']],
    on=['playername', 'position'], how='left'
).fillna(0).merge(
    carry_data[['playername', 'carry_score']], on='playername', how='left'
).fillna(0.5)

ranking_final['Score_Final'] = ranking_final.apply(
    lambda row: (
        row['Score'] * pesos_role_scores[row['position']]['score_jogo'] * 0.95 +
        row['carry_score'] * pesos_role_scores[row['position']]['carry_score'] * 0.95 +
        row['consistency_score'] * pesos_role_scores[row['position']]['consistency_score'] * 0.95 +
        row['AnchorScore'] * 0.05
    ), axis=1
)

# Ranking por role
for role in ['top', 'jng', 'mid', 'bot', 'sup']:
    df_role = ranking_final[ranking_final['position'] == role].copy()
    print(f"\n{'='*65}")
    print(f"ROLE: {role.upper()}")
    print(f"{'='*65}")
    print(df_role[['playername', 'teamname', 'W', 'Score', 'carry_score', 'AnchorScore', 'consistency_score', 'Score_Final']]
          .sort_values('Score_Final', ascending=False)
          .reset_index(drop=True)
          .assign(rank=lambda x: range(1, len(x)+1))
          .set_index('rank')
          .round(3)
          .to_string())

# Team Power Ranking
print(f"\n{'='*65}")
print(f"TEAM POWER RANKING — {LIGA}")
print(f"{'='*65}")
team_ranking = ranking_final.groupby('teamname').agg(
    Score_Final=('Score_Final', 'mean'),
    W=('W', 'mean')
).reset_index().sort_values('Score_Final', ascending=False).reset_index(drop=True)
team_ranking.index += 1
print(team_ranking.round(3).to_string())


ROLE: TOP
     playername         teamname      W  Score  carry_score  AnchorScore  consistency_score  Score_Final
rank                                                                                                    
1           fNb       RED Canids  0.629  0.547        0.628        0.699              0.858        0.643
2          xyno             LOUD  0.678  0.542        0.714        0.861              0.734        0.639
3          Zest              LØS  0.563  0.547        0.675        0.764              0.711        0.623
4         Guigo            FURIA  0.752  0.532        0.622        0.519              0.603        0.567
5          Boal  Vivo Keyd Stars  0.401  0.477        0.503        0.580              0.356        0.461
6          Robo      paiN Gaming  0.313  0.484        0.500        0.674              0.282        0.451
7        Devost         Leviatan  0.511  0.452        0.518        0.506              0.356        0.447
8         curty        Fluxo W7M  0.213  0.4

In [ ]:
# ============================================================
# SCORE 50-100 E TIER
# ============================================================
# Normaliza Score_Final para 50-100
min_s = ranking_final['Score_Final'].min()
max_s = ranking_final['Score_Final'].max()
ranking_final['Score_100'] = 50 + (ranking_final['Score_Final'] - min_s) / (max_s - min_s + 0.01) * 50

# Top 3 e bottom 3 absolutos (cross-role)
top3 = ranking_final.nlargest(3, 'Score_Final').index
bot3 = ranking_final.nsmallest(3, 'Score_Final').index

# Thresholds para os demais baseados em desvio padrão
resto = ranking_final[~ranking_final.index.isin(top3) & ~ranking_final.index.isin(bot3)]
media = resto['Score_100'].mean()
std = resto['Score_100'].std()

def atribuir_tier(row):
    if row.name in top3:
        return 'S+'
    if row.name in bot3:
        return 'F'
    s = row['Score_100']
    if s >= media + std:
        return 'S'
    elif s >= media + std * 0.3:
        return 'A'
    elif s >= media - std * 0.3:
        return 'B'
    else:
        return 'C'

ranking_final['Tier'] = ranking_final.apply(atribuir_tier, axis=1)

# Exibe com tier
for role in ['top', 'jng', 'mid', 'bot', 'sup']:
    df_role = ranking_final[ranking_final['position'] == role].copy()
    print(f"\n{'='*65}")
    print(f"ROLE: {role.upper()}")
    print(f"{'='*65}")
    print(df_role[['playername', 'teamname', 'W', 'Score_100', 'Tier']]
          .sort_values('Score_100', ascending=False)
          .reset_index(drop=True)
          .assign(rank=lambda x: range(1, len(x)+1))
          .set_index('rank')
          .round(1)
          .to_string())

print(f"\n{'='*65}")
print("DISTRIBUIÇÃO DE TIERS")
print(f"{'='*65}")
print(ranking_final['Tier'].value_counts().sort_index().to_string())


ROLE: TOP
     playername         teamname    W  Score_100 Tier
rank                                                 
1           fNb       RED Canids  0.6       89.8   S+
2          xyno             LOUD  0.7       89.3   S+
3          Zest              LØS  0.6       87.1    S
4         Guigo            FURIA  0.8       79.8    A
5          Boal  Vivo Keyd Stars  0.4       65.8    C
6          Robo      paiN Gaming  0.3       64.6    C
7        Devost         Leviatan  0.5       64.1    C
8         curty        Fluxo W7M  0.2       55.4    C

ROLE: JNG
     playername         teamname    W  Score_100 Tier
rank                                                 
1          Tatu            FURIA  0.8       98.7   S+
2     Drakehero              LØS  0.6       72.0    B
3         Curse       RED Canids  0.6       71.7    B
4      YoungJae             LOUD  0.7       70.1    B
5        CarioK      paiN Gaming  0.3       68.0    C
6       Disamis  Vivo Keyd Stars  0.4       67.6    C
7     